<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Graph_Neural_Networks_(GNNs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch_geometric

In [ ]:
import torch
from torch_geometric.nn import GCNConv

class SimpleGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Example graph data
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 2, 0], [1, 0, 1, 2]], dtype=torch.long)
x = torch.tensor([[1], [2], [3]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

model = SimpleGNN(input_dim=1, hidden_dim=4, output_dim=2)
output = model(data)
print(output)